In [12]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [13]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
assets = [:USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net    │
├────────────┼────────────┼──────────────┼─────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │   0.0246253 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │   0.0560778 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │   0.0302388 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │   0.0125024 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │  -0.0201209 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │ -0.00119203 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │     0.01818 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │  -0.0566908 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │   -0.042229 │
│

In [14]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))

417×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2022-09-30
┌────────────┬─────────────┬──────────────┬─────────────┐
│            │ USA_Net     │ World_ex_USA │ EM_Net      │
├────────────┼─────────────┼──────────────┼─────────────┤
│ 1988-01-31 │   0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │   0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │  -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │  0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │  0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │   0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │  -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │   -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │      ⋮      │      ⋮       │      ⋮      │
│ 2022-03-31 │     0.03417 │    0.0115373 │  -0.0228443 │
│ 2022-04-30 │  -0.0952756 │   -0.0679247 │  -0.0572341 │
│ 2022-05-31 │ -0.00270982 │    0.0082682 │   0.0043937 │
│ 2022-06-30 │  -0.0868359 │   -0.0988188 │  -0.068761

In [15]:
[assets; :ACWI_Net]

4-element Vector{Symbol}:
 :USA_Net
 :World_ex_USA
 :EM_Net
 :ACWI_Net

In [34]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1507,1.246,0.0676,0.4027,-2.257,-0.84,-0.2075,0.0703,1.6368,0.4485,0.1412
World_ex_USA,0.1653,1.4555,0.0499,0.4126,-2.7986,-0.913,-0.2363,0.1497,2.4191,0.3017,0.1038
EM_Net,0.233,1.893,0.0684,0.5702,-4.1,-1.0973,-0.3213,0.1423,3.896,0.2936,0.1038


In [69]:
hmm = hmm_est(period_returns, 3)
regime_summary(hmm[1], assets, 12)


Means
Standard deviations
Regimes transition matrix


Regime,USA_Net,World_ex_USA,EM_Net
1,-0.041,-0.076,-0.062
2,0.219,0.304,0.398
3,-0.025,-0.253,-0.409


Regime,USA_Net,World_ex_USA,EM_Net
1,0.201,0.216,0.302
2,0.078,0.081,0.114
3,0.094,0.059,0.116


"Correlations in regime 1"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.889,0.801
World_ex_USA,0.889,1.000,0.842
EM_Net,0.801,0.842,1.000


"Correlations in regime 2"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.768,0.492
World_ex_USA,0.768,1.000,0.667
EM_Net,0.492,0.667,1.000


"Correlations in regime 3"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.847,0.657
World_ex_USA,0.847,1.000,0.775
EM_Net,0.657,0.775,1.000


Regime,1,2,3
1,0.921,0.041,0.038
2,0.000,0.775,0.225
3,0.185,0.472,0.343


In [70]:
n_assets = length(assets)
scenarios = simulate_hmm(hmm[1], n_assets, 180, 100_000)

for a in 1:n_assets
    print_percentiles(scenarios[a,:,:], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12, string(assets[a]))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.3999,-0.3534,-0.3366,-0.3309,-0.3354,-0.3312,-0.3334,-0.3313,-0.3326,-0.333,-0.3356,-0.3332,-0.3329,-0.333,-0.3323
0.05,-0.3119,-0.2601,-0.245,-0.2401,-0.2392,-0.2374,-0.2397,-0.238,-0.2396,-0.2392,-0.2404,-0.2408,-0.2392,-0.2414,-0.24
0.25,-0.1087,-0.0421,-0.0259,-0.022,-0.0199,-0.0203,-0.0208,-0.0195,-0.0205,-0.0204,-0.0206,-0.0214,-0.0197,-0.0214,-0.0206
0.5,0.0234,0.0843,0.095,0.0981,0.0975,0.0978,0.0982,0.0995,0.0982,0.0983,0.0985,0.0973,0.0989,0.0977,0.0992
0.75,0.1419,0.1812,0.1868,0.1884,0.1883,0.1885,0.1883,0.1898,0.1879,0.1883,0.1883,0.1878,0.1884,0.1874,0.1892
0.95,0.303,0.3007,0.3024,0.3027,0.302,0.3027,0.3018,0.3023,0.3017,0.302,0.3021,0.3024,0.3024,0.3006,0.3039
0.98,0.3708,0.3533,0.3526,0.3531,0.351,0.3546,0.3523,0.3541,0.3531,0.3517,0.3515,0.3527,0.3543,0.3514,0.3547


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.4656,-0.4145,-0.3983,-0.3965,-0.3978,-0.3927,-0.3972,-0.3926,-0.3981,-0.3965,-0.3962,-0.3958,-0.3957,-0.3977,-0.3933
0.05,-0.3728,-0.316,-0.2995,-0.2961,-0.2979,-0.2929,-0.295,-0.2922,-0.2997,-0.2946,-0.2963,-0.298,-0.2947,-0.2983,-0.2951
0.25,-0.1525,-0.0822,-0.0648,-0.0596,-0.059,-0.0579,-0.0585,-0.0566,-0.0589,-0.0585,-0.0584,-0.0587,-0.0577,-0.0601,-0.0572
0.5,-0.0069,0.0641,0.0776,0.0815,0.0821,0.0822,0.082,0.0838,0.0818,0.0816,0.0824,0.0817,0.0827,0.0813,0.083
0.75,0.1272,0.1825,0.1913,0.1934,0.1932,0.1933,0.1942,0.1949,0.1931,0.1937,0.1939,0.1936,0.1933,0.1926,0.1952
0.95,0.305,0.322,0.3261,0.326,0.3272,0.3269,0.3273,0.3263,0.3264,0.328,0.3272,0.3274,0.3261,0.3252,0.3276
0.98,0.3807,0.3783,0.3804,0.3803,0.3801,0.3789,0.3796,0.3799,0.3787,0.3791,0.3805,0.3803,0.3802,0.3793,0.3798


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.6134,-0.5499,-0.5319,-0.5237,-0.5266,-0.5237,-0.5289,-0.5195,-0.5273,-0.5271,-0.5242,-0.5287,-0.5229,-0.5263,-0.5237
0.05,-0.4875,-0.4145,-0.3954,-0.3915,-0.3902,-0.3875,-0.3923,-0.3866,-0.3918,-0.3895,-0.39,-0.3928,-0.3898,-0.3913,-0.387
0.25,-0.1889,-0.1058,-0.0854,-0.0819,-0.0801,-0.0793,-0.0811,-0.0778,-0.0806,-0.0788,-0.0815,-0.0811,-0.0794,-0.0806,-0.0786
0.5,0.0079,0.0842,0.0992,0.1031,0.1028,0.1051,0.104,0.105,0.1033,0.1042,0.1041,0.1045,0.1038,0.1023,0.1057
0.75,0.1929,0.245,0.2548,0.2553,0.2566,0.2564,0.2574,0.2576,0.2562,0.2577,0.2563,0.256,0.2571,0.256,0.2569
0.95,0.4438,0.4431,0.4475,0.4477,0.4468,0.4459,0.4477,0.448,0.4437,0.4469,0.4467,0.4464,0.445,0.4467,0.4474
0.98,0.5507,0.5267,0.5289,0.5261,0.5253,0.5231,0.5265,0.527,0.522,0.526,0.5226,0.5239,0.5246,0.5268,0.5277


In [71]:
y_data = annualise(scenarios[1, :, 61:73],12)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

"mean"

0.07374883373683425

"std"

0.165463074431944

"skewness"

-0.6761395434471701

"kurtosis"

0.6797054354900056

In [72]:
weights = [0.321, 0.67, 0.009]
#weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(acwi, [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0.02,-0.4268,-0.3827,-0.3708,-0.3658,-0.3684,-0.3655,-0.3639,-0.362,-0.366,-0.3639,-0.3628,-0.3638,-0.3632,-0.3648
0.05,-0.3371,-0.2864,-0.2753,-0.2706,-0.2705,-0.2654,-0.2703,-0.268,-0.2706,-0.2697,-0.2693,-0.2703,-0.2682,-0.2691
0.25,-0.1259,-0.0643,-0.0487,-0.0429,-0.0427,-0.0417,-0.0433,-0.0403,-0.042,-0.0419,-0.0413,-0.0416,-0.0416,-0.042
0.5,0.0132,0.0749,0.0851,0.0886,0.0885,0.0893,0.0888,0.0902,0.0883,0.0891,0.0889,0.0892,0.0899,0.0891
0.75,0.1385,0.1818,0.1871,0.1896,0.1898,0.1896,0.1899,0.1908,0.189,0.1899,0.1901,0.1902,0.1895,0.1892
0.95,0.2982,0.3079,0.3115,0.311,0.3114,0.311,0.3117,0.3103,0.3093,0.3109,0.3098,0.3095,0.309,0.3091
0.98,0.3641,0.3595,0.361,0.3597,0.3613,0.3609,0.3607,0.36,0.3597,0.3589,0.3611,0.3591,0.3591,0.3589


In [73]:
display("mean")
display(mean(annualise(acwi[:,60:72],12)))
display("std")
display(std(annualise(acwi[:,60:72],12)))
display("skewness")
display(skewness(annualise(acwi[:,60:72],12)))
display("kurtosis")
display(kurtosis(annualise(acwi[:,60:72],12)))

"mean"

0.06360769501319016

"std"

0.1772543851441384

"skewness"

-0.6376396508191977

"kurtosis"

0.42225630462892916

In [74]:
ACWI_period_returns = returns_usd[:ACWI_Net] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(ACWI_period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1562,1.339,0.0595,0.3651,-2.65,-0.9712,-0.2175,0.122,2.553,0.381,0.1197


In [75]:
M = [1 2 1; 3 4 1; 5 6 1]
cov(M')

3×3 Matrix{Float64}:
 0.333333  0.666667  1.0
 0.666667  2.33333   4.0
 1.0       4.0       7.0

In [76]:
M2 = [1 2 1; 3 4 1; 5 6 missing]
cov(M2')

MethodError: MethodError: Cannot `convert` an object of type Missing to an object of type Float64

Closest candidates are:
  convert(::Type{Float64}, !Matched::Measures.AbsoluteLength)
   @ Measures C:\Users\matsz\.julia\packages\Measures\PKOxJ\src\length.jl:12
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex}
   @ DualNumbers C:\Users\matsz\.julia\packages\DualNumbers\5knFX\src\dual.jl:24
  convert(::Type{T}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S}
   @ Ratios C:\Users\matsz\.julia\packages\Ratios\FsiCW\src\Ratios.jl:51
  ...
